# Road Rage: Finding the Ideal Speed Limit

### Assumptions
* Drivers want to go up to 120 km/hr.
* The average car is 5 meters long.
* Drivers want at least a number of meters equal to their speed in meters/second between them and the next car.
* Drivers will accelerate 2 m/s<sup>2</sup> up to their desired speed as long as they have room to do so.
* If another car is too close, drivers will match that car's speed until they have room again.
* If a driver would hit another car by continuing, they stop.
* Drivers will randomly (10% chance each second) slow by 2 m/s.
* This section of road is one lane going one way.
* Assume that drivers enter the road at the speed they left.
* Simulation starts with 30 cars per kilometer, evenly spaced.

## Normal Mode
We have a 1 kilometer section of road being built and do not know what the speed limit should be. This notebook simulates the 1 kilometer of road. Even though this road is not circular, the simulation treats it as such in order to generate a continuous flow of traffic.

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random
from traffic_lib import *
%matplotlib inline

In [2]:
class Car:
    """
    Responsibilities:
    - Know speed (in m/s)
    - Know distance to driver ahead
    - Keep distance from driver ahead
    - Accelerate if possible
    - Match speed of those ahead if within safety zone
    - Stop if new location would result in crash (0 distance to car ahead)
    """
    
    def __init__(self, location, gap=28, speed_limit=33, start_speed=28):
        # speed limit is in km/hr = 1000 m / 3600 s = 1/3.6 m/s;
        self.desired_speed = speed_limit
        self.speed = start_speed
        # Initial speed of 28 m/s 
        self.gap = gap
        self.size = 5
        self.location = location
        self.advance = None
    
    def drive(self, car_ahead):
        self.advance = self.speed - car_ahead.speed

        if car_ahead.location - (car_ahead.size - 1) - self.location > self.location - car_ahead.location:
            self.gap = car_ahead.location - (car_ahead.size - 1) - self.location
        else:
            self.gap = car_ahead.location + 1000 - (car_ahead.size - 1) - self.location

        if self.stop():
            return self
        elif self.match_speed(car_ahead):
            return self
        elif self.random_slowdown():
            return self
        elif self.accelerate():
            return self

    def stop(self, car_ahead):
        if self.gap - self.advance <= 0:
            self.speed = 0
            self.location = car_ahead.location - (car_ahead.size - 1)
            self.update_location()
            # Needs special location update; stops at 0 gap
            return True
        
    def match_speed(self, car_ahead):
        if self.gap - self.advance <= self.speed:
            self.speed = car_ahead.speed
            self.update_location()
            return True
        
    def random_slowdown(self):
        if random.random(0, 1) < 0.1:
            self.speed -= 2
            self.update_location()
            return True
    
    def accelerate(self):
        if self.speed < self.desired_speed:
            self.speed += 2
            self.update_location()
            return True
    
    def update_location(self):
        self.location = (self.location + self.speed) % 1000

In [6]:
class Road:
    """
    Responsibilities:
    - Hold length of road
    - Keep a list of vehicles on road
        - Initialize with number of cars
        - 1000 - sum(vehicle.size) // len(self.vehicles)
    - Hold potential for slowdown
    
    Collaborators:
    - Car
    """
    def __init__(self, num_trucks=0):
        self.meter_marker = np.repeat(0, 1000)
        # May not need this; just keep array of car objects and the position of the front of their car
        self.total_vehicle_space = (30 - num_trucks) * 5 + num_trucks * 25
        self.initial_gap = 1000 - self.total_vehicle_space // 30
        self.initial_location
        self.vehicles = [Car((4 + initial_gap)*n, initial_gap) for n in range(30 - num_trucks)]
        [self.vehicles.append(Truck() for _ in range(num_trucks))]
        # Needs refactoring; need to evenly space. Perhaps make Car have a chance of being a truck?
        # They are long enough where they won't cause a collision on initialization
        # Could initialize vehicles and update location before starting simulation
        
        self.vehicles[-1].gap = 1000 - self.vehicles[-1].location
        
    def reindex_car(self):
        if self.vehicles[-1].location < self.total_vehicle_space:
            self.index(0, self.vehicles.pop(-1))
            return True
        
        
    


In [4]:
class Simulation:
    """
    Responsibilities:
    - Have road place cars at beginning of road when they reach the end
    - Keep track of time (seconds)
    - Step through time (ticks)
        - For each car on Road, tell car behind new situation and allow it to react
        - Update locations; location % 1000
    - Report stats; return traffic jam status
    - Pop, Append
    
    Collaborators:
    - Car
    - Road
    """
    
    def __init__(self):
        self.road = Road()
        self.ticks = 0
        self.stopped_traffic = False
    
    def iterate(self):
        """
        Reverse list to process cars at the end of the road first.
        n.b., lower numbers are physically ahead of higher numbers.
        """
        for idx, v in enumerate(self.road.vehicles[::-1]):
            v.drive(self.road.vehicles(idx - 1))
            
    def run_sim(self, duration=6):
        while self.ticks < duration:
            self.iterate()
            ticks += 1        